In [13]:
import numpy as np
import xarray as xr
import netCDF4 as nc
import model_crommelin_seasonal
import feature_crommelin 
from importlib import reload
import sys 
import os
from os import mkdir
from os.path import join,exists
from importlib import reload


In [2]:
import matplotlib.pyplot as plt 
%matplotlib inline

Create directories to save results

In [3]:
topic_dir = "/scratch/jf4241/crommelin"
if not exists(topic_dir): mkdir(topic_dir)
day_dir = join(topic_dir,"2022-07-22")
if not exists(day_dir): mkdir(day_dir)
exp_dir = join(day_dir,"1")
if not exists(exp_dir): mkdir(exp_dir)
ra_dir = join(exp_dir,"reanalysis_data")
if not exists(ra_dir): mkdir(ra_dir)
ra_dir_contiguous = join(ra_dir,"contiguous") # For long, unbroken record
if not exists(ra_dir_contiguous): mkdir(ra_dir_contiguous)
ra_dir_seasonal = join(ra_dir,"seasonal") # For data split into seasons
if not exists(ra_dir_seasonal): mkdir(ra_dir_seasonal)
hc_dir = join(exp_dir,"hindcast_data")
if not exists(hc_dir): mkdir(hc_dir)
featspec_dir = join(exp_dir,"featspec") # Metadata with info to compute features
if not exists(featspec_dir): mkdir(featspec_dir)
results_dir = join(exp_dir,"results")
if not exists(results_dir): mkdir(results_dir)
results_dir_ra = join(results_dir,"ra")
if not exists(results_dir_ra): mkdir(results_dir_ra)
results_dir_hc = join(results_dir,"hc")
if not exists(results_dir_hc): mkdir(results_dir_hc)

Set physical parameters

In [4]:
dt_samp = 0.5 # Time step to save out
dt_szn = 0.74 # Time resolution for the seasonal model 
szn_start = 300.0
szn_length = 250.0
year_length = 400.0
Nt_szn = int(szn_length / dt_szn)
szn_avg_window = 5.0
burnin_time = 500.0

In [22]:
reload(model_crommelin_seasonal)

<module 'model_crommelin_seasonal' from '/home/jf4241/ecmwf/tpt_ecmwf/demo/model_crommelin_seasonal.py'>

Initialize the model.

In [23]:
fundamental_param_dict = dict({"b": 0.5, "beta": 1.25, "gamma_limits": [0.15, 0.22], "C": 0.1, "x1star": 0.95, "r": -0.801, "year_length": year_length})
crom = model_crommelin_seasonal.SeasonalCrommelinModel(fundamental_param_dict)


Create "reanalysis" in the file folder reserved for contiguous data.

In [24]:
traj_filename_ra = join(ra_dir_contiguous,"crom_long.nc")

In [25]:
# Run the model forward
x0 = np.zeros((1,7))
x0[0,6] = (1957 + 0.2)*fundamental_param_dict["year_length"]  # Starting time is about 20% of the way through 1957 (arbitrary)
dt_save = 0.5
tmax_save = 5*fundamental_param_dict["year_length"] + burnin_time
t_save = np.arange(0,tmax_save,dt_samp)
crom.integrate_and_save(x0,t_save,traj_filename_ra,burnin_time=burnin_time)

Integrated through time 1000.0000000001588 out of 2499.5
Integrated through time 2000.0999999992764 out of 2499.5


ValueError: multi-dimensional array attributes not supported

In [9]:
ds = xr.open_dataset(traj_filename_ra)

In [10]:
ds.attrs

{'b': 0.5,
 'beta': 1.25,
 'gamma_limits': array([0.15, 0.22]),
 'C': 0.1,
 'x1star': 0.95,
 'r': -0.801,
 'year_length': 400.0,
 'xstar': array([ 0.95   ,  0.     ,  0.     , -0.76095,  0.     ,  0.     ]),
 'alpha': array([0.24008435, 0.73437566]),
 'delta': array([ 0.38413496, -1.24278958]),
 'epsilon': 1.44050610585137}

In [11]:
ds.close()

In [ ]:
traj_filename_ra

Create "hindcast" data in the file folder reserved for hindcast data.

In [ ]:
# Generate hindcast dataset
t_abs_range = crom.q["year_length"]*np.array([1960,1970])
crom.generate_hindcast_dataset(
    traj_filename_ra,hc_dir,t_abs_range,dt_samp,
    ens_size=30,ens_duration=47,ens_gap=13,pert_scale=0.01
)

Extract features of interest from both the reanalysis and hindcast datasets. This will be expensive, as we have to read from a large database of files. Therefore, we should minimize the number of times we do this in development. Therefore, we should read in ALL possible features we MIGHT use for the downstream tasks of K-means clustering. Some extra reduction is likely necessary.

In [ ]:
# Extract all features of potential use from the reanalysis dataset. 
# Don't waste data by time-delaying, that's just silly
feat_crom = feature_crommelin.SeasonalCrommelinModelFeatures(
    szn_start,szn_length,year_length,Nt_szn,
    szn_avg_window,dt_samp
)

In [ ]:
# Load the already-executed model (contiguous)
X_ra_cont = xr.open_dataset(traj_filename_ra)["X"]
print(X_ra_cont.sel(feature="x1").isel(t_sim=0))
print(X_ra_cont.sel(feature="x2").isel(t_sim=0))

In [ ]:
# Plot x1 and x4 over time for 4 annual cycles
fig,ax = plt.subplots(ncols=2, figsize=(12,6))
xr.plot.plot(
    X_ra_cont.sel(feature='x1',member=0)
    .where(X_ra_cont['t_sim'] < 4*crom.q["year_length"], drop=True),
    x='t_sim', ax=ax[0]
)
xr.plot.plot(
    X_ra_cont.sel(feature='x4',member=0)
    .where(X_ra_cont['t_sim'] < 4*crom.q["year_length"], drop=True),
    x='t_sim', ax=ax[1]
)


In [ ]:
# Compute the climatology
feat_crom.compute_climatology(in_filename,save_filename)

In [ ]:
# Plot some hindcasts on top of climatology
